Copyright 2023 Ian Housman and Elizabeth Hardwick

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.


* Notebook to explore the relationships between change from NAIP-based change detection to tune Sentinel 2-based change detection

In [9]:
# Import modules
import geeViz.getImagesLib as gil
import  geeViz.foliumView as fv
import  geeViz.geeView as gv
import geeViz.gee2Pandas as g2p
import pandas as pd
import matplotlib.pyplot as plt
import numpy,seaborn,os,json
import tch_setup as tch
ee = fv.ee

# Can choose whether to use Folium or the original GEEView
# Folium sets up more quickly, but lacks the ablity to query map layers
# Map = fv.foliumMapper()
Map = gv.Map

Map.port = 1234
print('done')


done


In [14]:
# First bring in monitoring sites and set up some parameters
Map.clearMap()

monitoring_sites = ee.FeatureCollection('projects/gtac-lamda/assets/giant-sequoia-monitoring/Inputs/Trees_of_Special_Interest')

# Pull projection info from TCH outputs
projection = tch.tchC.first().projection().getInfo()
crs = projection['crs']
transform = projection['transform']
scale = None

# Set study area
studyArea = monitoring_sites.geometry().bounds(500,crs)

# Set up date params
preStartYear = 2019
preEndYear = 2020
postStartYear = 2022
postEndYear = 2022
startJulian = 210
endJulian = 230

output_dir = r'Q:\LAMDA_workspace\giant-sequoia-monitoring\outputs'

# Bands to pull for graphs
bandNames = ['blue','green','red','swir1','nir','NDVI','NBR','NDMI','brightness','greenness','wetness']

# Bands for change heuristics
changeBands = ['greenness','wetness','NBR']
dChangeThresholds = [-0.05,-0.02,-0.2]
zChangeThresholds = [-5]
changeReducer = ee.Reducer.mode()


######################################################################
Map.centerObject(studyArea)
Map.addLayer(monitoring_sites.map(lambda f:ee.Feature(f).buffer(10).bounds()),{'strokeColor':'FF0'},'Monitoring Sites')
Map.addLayer(studyArea,{'strokeColor':'F00','strokeWidth':10},'Study Area')
# # print(Map.mapArgs)
Map.turnOnInspector()
Map.view()

Adding layer: Monitoring Sites
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: C:\Users\ihousman/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd q:\LAMDA_Scripts_Repo\Sequoia_Monitoring


In [15]:

# Get S2 data and do simple two date and z-score based change detection
Map.clearMap()

# Get all S2 data for date ranges and then some for TDOM
s2s  = gil.getProcessedSentinel2Scenes(studyArea,preStartYear-1,postEndYear+1,startJulian,endJulian,\
         applyQABand = False,
         applyCloudScore = False,
         applyShadowShift = False,
         applyTDOM = True,
         cloudScoreThresh = 20,
         performCloudScoreOffset = True,
         cloudScorePctl = 10,
         cloudHeights = ee.List.sequence(500,10000,500),
         zScoreThresh = -1,
         shadowSumThresh = 0.35,
         contractPixels = 1.5,
         dilatePixels = 3.5,
         shadowSumBands = ['nir','swir1'],
         resampleMethod = 'near',
         toaOrSR = 'TOA',
         convertToDailyMosaics = False,
         applyCloudProbability = True,
         preComputedCloudScoreOffset = None,
         preComputedTDOMIRMean = None,
         preComputedTDOMIRStdDev = None,
         cloudProbThresh = 40)

# Get pre and post images
postS2s = s2s.filter(ee.Filter.calendarRange(postStartYear,postEndYear,'year'))
preS2s = s2s.filter(ee.Filter.calendarRange(preStartYear,preEndYear,'year'))

# Compute pre and post composites
postComp = postS2s.select(bandNames).median()
preComp = preS2s.select(bandNames).median()

# Compute stats for z-based change
preMean = preS2s.select(bandNames).mean()
preStdDev = preS2s.select(bandNames).reduce(ee.Reducer.stdDev())

# Simple difference and z score
compDiff = postComp.subtract(preComp)
postZ = postComp.subtract(preMean).divide(preStdDev)

# Threshold simple difference and z
# Based on: green to red = diff greenness <-0.05 and diff wetness < -0.02 and diff nbr < -0.2
dChange = compDiff.select(changeBands).lte(dChangeThresholds).reduce(changeReducer)
zChange = postZ.select(changeBands).lte(zChangeThresholds).reduce(changeReducer)


Map.addLayer(preComp,gil.vizParamsFalse,'S2 Median {}-{}'.format(preStartYear,preEndYear),False)
Map.addLayer(postComp,gil.vizParamsFalse,'S2 Median {}-{}'.format(postStartYear,postEndYear),False)

Map.addLayer(compDiff.select(['brightness','greenness','wetness']),{'min':-0.2,'max':0.2},'TC-Diff-Comp',False)
Map.addLayer(postZ.select(['brightness','greenness','wetness']),{'min':-5,'max':5},'TC-Z-Comp',False)

Map.addLayer(dChange.selfMask(),{'palette':'DD0','classLegendDict':{'D-Loss':'DD0'}},'D-Loss',False)
Map.addLayer(zChange.selfMask(),{'palette':'D00','classLegendDict':{'Z-Loss':'D00'}},'Z-Loss',False)

# Bring in TCH data
tchStack = tch.tchC.filter(ee.Filter.stringContains('system:index','v3shadows_extract_gfch'))\
    .sort('startYear')\
    .toBands()

for yr1,yr2 in tch.tch_year_pairs:
    tchImg = tch.getTCHExtract(yr1,yr2,filterString= 'v3shadows_extract_gfch')
    Map.addLayer(tchImg,tch.extractedViz,'TCH Extract {}-{}'.format(yr1,yr2),False)

Map.addLayer(monitoring_sites.map(lambda f:ee.Feature(f).buffer(10).bounds()),{'strokeColor':'FF0'},'Monitoring Sites')
Map.addLayer(studyArea,{'strokeColor':'F00','strokeWidth':10},'Study Area')
Map.centerObject(studyArea)
Map.turnOnInspector()
Map.view()


Get Processed Sentinel2: 
Start date: Jul 29 2018 , End date: Aug 18 2023
startYear :  2018
endYear :  2023
startJulian :  210
endJulian :  230
applyQABand :  False
applyCloudScore :  False
applyShadowShift :  False
applyTDOM :  True
performCloudScoreOffset :  True
cloudScorePctl :  10
zScoreThresh :  -1
shadowSumBands :  ['nir', 'swir1']
resampleMethod :  near
convertToDailyMosaics :  False
applyCloudProbability :  True
preComputedCloudScoreOffset :  None
preComputedTDOMIRMean :  None
preComputedTDOMIRStdDev :  None
origin :  Sentinel2
wrapOffset :  0
cloudProbThresh :  40
cloudScoreThresh :  20
toaOrSR :  TOA
Using S2 Collection: COPERNICUS/S2_HARMONIZED
Joining pre-computed cloud probabilities from: COPERNICUS/S2_CLOUD_PROBABILITY
Apply Cloud Probability
Applying TDOM
Computing irMean for TDOM
Computing irStdDev for TDOM
Adding layer: S2 Median 2019-2020
Adding layer: S2 Median 2022-2022
Adding layer: TC-Diff-Comp
Adding layer: TC-Z-Comp
Adding layer: D-Loss
Adding layer: Z-Loss
Add

In [7]:
Map.clearMap()
preBns = preComp.bandNames().map(lambda bn:ee.String(bn).cat('_Comp_yr{}-{}_jd{}-{}'.format(preStartYear,preEndYear,startJulian,endJulian)))
postBns = postComp.bandNames().map(lambda bn:ee.String(bn).cat('_Comp_yr{}-{}_jd{}-{}'.format(postStartYear,postEndYear,startJulian,endJulian)))
diffBns = compDiff.bandNames().map(lambda bn:ee.String(bn).cat('_CompDiff_yrs{}-{}__{}-{}_jd{}-{}'.format(preStartYear,preEndYear,postStartYear,postEndYear,startJulian,endJulian)))
zBns = postZ.bandNames().map(lambda bn:ee.String(bn).cat('_Z_yrs{}-{}__{}-{}_jd{}-{}'.format(preStartYear,preEndYear,postStartYear,postEndYear,startJulian,endJulian)))
stack = preComp.rename(preBns)\
    .addBands(postComp.rename(postBns))\
    .addBands(compDiff.rename(diffBns))\
    .addBands(postZ.rename(zBns))\
    .addBands(tchStack)
    # .addBands(change)
print(stack.bandNames().getInfo())
startNSamples=1000000
maxNSamples=5000
random_sample =ee.FeatureCollection.randomPoints(studyArea, startNSamples, 0, 50)
random_sample = tchStack.select([1]).mask().selfMask().reduceRegions(\
      random_sample,\
      ee.Reducer.first(), \
      scale, \
      crs,\
      transform, \
      4).filter(ee.Filter.notNull(['first'])).limit(maxNSamples)
# final_sampleSize = random_sample.size().getInfo()
# print(final_sampleSize.size().getInfo())
Map.addLayer(random_sample.map(lambda f:ee.Feature(f).buffer(10).bounds()),{},'Sample')


monitoring_sites_out_csv = os.path.join(output_dir,'Giant_Sequoia_Monitoring_Sites_Change_yrs{}-{}--{}-{}_jd{}-{}.csv'.format(preStartYear,preEndYear,postStartYear,postEndYear,startJulian,endJulian))
random_sample_out_csv = os.path.join(output_dir,'Giant_Sequoia_Random_Sample_n{}_Change_yrs{}-{}--{}-{}_jd{}-{}.csv'.format(maxNSamples,preStartYear,preEndYear,postStartYear,postEndYear,startJulian,endJulian))

g2p.geeToLocalZonalStats(monitoring_sites,stack,monitoring_sites_out_csv,reducer=ee.Reducer.first(),scale=scale,crs=crs,transform=transform,tileScale=4,overwrite=False,maxNumberOfFeatures=5000)
g2p.geeToLocalZonalStats(random_sample,stack,random_sample_out_csv,reducer=ee.Reducer.first(),scale=scale,crs=crs,transform=transform,tileScale=4,overwrite=False,maxNumberOfFeatures=5000)
# Map.view()

['blue_Comp_yr2019-2020_jd210-230', 'green_Comp_yr2019-2020_jd210-230', 'red_Comp_yr2019-2020_jd210-230', 'swir1_Comp_yr2019-2020_jd210-230', 'nir_Comp_yr2019-2020_jd210-230', 'NDVI_Comp_yr2019-2020_jd210-230', 'NBR_Comp_yr2019-2020_jd210-230', 'NDMI_Comp_yr2019-2020_jd210-230', 'brightness_Comp_yr2019-2020_jd210-230', 'greenness_Comp_yr2019-2020_jd210-230', 'wetness_Comp_yr2019-2020_jd210-230', 'blue_Comp_yr2022-2022_jd210-230', 'green_Comp_yr2022-2022_jd210-230', 'red_Comp_yr2022-2022_jd210-230', 'swir1_Comp_yr2022-2022_jd210-230', 'nir_Comp_yr2022-2022_jd210-230', 'NDVI_Comp_yr2022-2022_jd210-230', 'NBR_Comp_yr2022-2022_jd210-230', 'NDMI_Comp_yr2022-2022_jd210-230', 'brightness_Comp_yr2022-2022_jd210-230', 'greenness_Comp_yr2022-2022_jd210-230', 'wetness_Comp_yr2022-2022_jd210-230', 'blue_CompDiff_yrs2019-2020__2022-2022_jd210-230', 'green_CompDiff_yrs2019-2020__2022-2022_jd210-230', 'red_CompDiff_yrs2019-2020__2022-2022_jd210-230', 'swir1_CompDiff_yrs2019-2020__2022-2022_jd210-230'

In [8]:
import plotly.express as px
t = pd.read_csv(random_sample_out_csv)
# display(t)
# plt.ion()
category_col = 'TCH_SEQU_2020_2022_v3shadows_extract_gfch_TCH_Count'
t=t[t[category_col]!=4]
seaborn.set_palette(['#00BFA5','#FF834C','#968B83','#F20']) #seaborn.color_palette("husl", 8))
# print(t.columns)

# seaborn.countplot(x=t[category_col])
# columns = t.columns
# waterColumn = 'Water_Transition_{}-{}'.format(preYear,postYear)
# print(t[waterColumn])
for plotBandName in ['NBR','brightness','greenness','wetness']:
    diffColumn = '{}_CompDiff_yrs{}-{}__{}-{}_jd{}-{}'.format(plotBandName,preStartYear,preEndYear,postStartYear,postEndYear,startJulian,endJulian)
    zColumn = '{}_Z_yrs{}-{}__{}-{}_jd{}-{}'.format(plotBandName,preStartYear,preEndYear,postStartYear,postEndYear,startJulian,endJulian)
# #     print(diffColumn)
#     t.hist(column = diffColumn, bins = 40, color = 'teal', alpha = 0.5)

# plt.show()
#     preColumn = '{}_yr{}_jd{}-{}'.format(plotBandName,preYear,startJulian,endJulian)
#     postColumn = '{}_yr{}_jd{}-{}'.format(plotBandName,postYear,startJulian,endJulian)
   
#     # display(t)
    
    

#     # fig, ax = plt.subplots()
#     seaborn.lmplot(x=preColumn,y=postColumn,data=t,hue=waterColumn,scatter_kws={'alpha':0.1})
#     # scat = ax.scatter(t[preColumn],t[postColumn],c=t[waterColumn],alpha=0.1)
#     # ax.set_xlabel(preColumn)
#     # ax.set_ylabel(postColumn)
#     # plt.legend(loc='upper left')                                                                                                                                                                                                                                                                                                                                                                                                             
#     # plt.show()

    # fig,ax=plt.subplots()
    # seaborn.violinplot(x =category_col,y =diffColumn,data = t,ax=ax)
    # fig,ax=plt.subplots()
    # seaborn.violinplot(x =category_col,y =zColumn,data = t,ax=ax)
    
    
# print(t.columns)

    display(px.histogram(data_frame=t, color=category_col, nbins=100,x=diffColumn,barmode='overlay',histnorm='probability density'))
    # display(px.histogram(data_frame=t, color=category_col, nbins=100,x=zColumn,barmode='overlay'))

In [5]:
out_json = os.path.splitext(out_csv)[0]+'.json'
o = open(out_json,'r')
fc = ee.FeatureCollection(json.load(o))
o.close()
print(fc.first().toDictionary().getInfo())
monitoring_sites_change = fc.filter(ee.Filter.eq('Decrease_NBR',1))
print(fc.aggregate_histogram('Decrease_NBR').getInfo())

Map.addLayer(monitoring_sites_change.map(lambda f:ee.Feature(f).buffer(10).bounds()),{'strokeColor':'0FF'},'Change Sites')
Map.view()

NameError: name 'out_csv' is not defined